In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import pickle
from math import floor
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dense, Dropout, Flatten, MaxPooling2D, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import History 
from tensorflow.keras import Model
from tensorflow.keras import backend as K
if K.backend() == 'tensorflow':
    import tensorflow as tf

In [66]:
train_path = './seg_train/seg_train/'
test_path = './seg_test/seg_test/'
Labels = ['buildings', 'forest','glacier','mountain','sea','street']
#Labels = ['buildings', 'forest','glacier','mountain','sea']
label_dict = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea', 5:'street'}
#label_dict = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea'}

In [67]:
min_count = len(os.listdir(train_path+'buildings'))*3

In [68]:
def get_data(path):
    images = list()
    lbls = list()
    for label in Labels:
        count = 0
        for image in os.listdir(path+label):
            if count < min_count:
                img = cv2.imread(path+label+r'/'+image)
                img = Image.fromarray(img , 'RGB')
                img = img.resize((50,50))
                images.append(np.array(img)/255)
                if label == 'buildings':
                    lbl = 0
                elif label == 'forest':
                    lbl = 1
                elif label == 'glacier':
                    lbl = 2
                elif label == 'mountain':
                    lbl = 3
                elif label == 'sea':
                    lbl = 4
                else:
                    lbl = 5
                lbls.append(lbl)
                if path == train_path:
                    rotated = img.rotate(45)
                    flipped = np.fliplr(img)
                    images.append(np.array(rotated)/255)
                    images.append(np.array(flipped)/255)
                    lbls.append(lbl)
                    lbls.append(lbl)
                    #train_label_count[label] += 3
                    count += 3
                
    images = np.array(images)
    lbls = np.array(lbls)
    
    return shuffle(images, lbls, random_state=42)

In [69]:
train_images, train_labels = get_data(train_path)

In [70]:
test_images, test_labels = get_data(test_path)

In [71]:
monitor = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [72]:
train_A, train_B = train_images, train_images

In [73]:
input_A = Input(shape=train_A.shape[1:], name='input_A')
dense = Dense(50, activation='relu')(input_A)
drop_1A = Dropout(0.3)(dense)
flat_A = Flatten()(drop_1A)
input_B = Input(shape=train_B.shape[1:], name='input_B')
conv_1 = Conv2D(filters=128,kernel_size=2,padding="same",activation="relu")(input_B)
pool_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)
conv_2 = Conv2D(filters=64,kernel_size=2,padding="same",activation="relu")(pool_1)
pool_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)
conv_3 = Conv2D(filters=32,kernel_size=2,padding="same",activation="relu")(pool_2)
pool_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)
drop_1 = Dropout(0.2)(pool_3)
flat_2 = Flatten()(drop_1)
hidden1 = Dense(500, activation='relu')(flat_2)
drop_2 = Dropout(0.2)(hidden1)
concat = Concatenate()([flat_A, drop_2])
output = Dense(6, activation="softmax")(concat)
model = Model(inputs=[input_A, input_B], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_B (InputLayer)            [(None, 50, 50, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 50, 50, 128)  1664        input_B[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_37 (MaxPooling2D) (None, 25, 25, 128)  0           conv2d_43[0][0]                  
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 25, 25, 64)   32832       max_pooling2d_37[0][0]           
___________________________________________________________________________________________

In [74]:
model.fit([train_A, train_B], train_labels, epochs=50,validation_split=.3, callbacks=[monitor], shuffle=True)

Train on 27606 samples, validate on 11832 samples
Epoch 1/50
27606/27606 [==============================] - 195s 7ms/sample - loss: 1.0460 - accuracy: 0.6204 - val_loss: 0.8008 - val_accuracy: 0.7087
Epoch 2/50
27606/27606 [==============================] - 175s 6ms/sample - loss: 0.7571 - accuracy: 0.7258 - val_loss: 0.7110 - val_accuracy: 0.7440
Epoch 3/50
27606/27606 [==============================] - 164s 6ms/sample - loss: 0.6587 - accuracy: 0.7616 - val_loss: 0.6869 - val_accuracy: 0.7495
Epoch 4/50
27606/27606 [==============================] - 165s 6ms/sample - loss: 0.5937 - accuracy: 0.7846 - val_loss: 0.6958 - val_accuracy: 0.7545
Epoch 5/50
27606/27606 [==============================] - 165s 6ms/sample - loss: 0.5434 - accuracy: 0.8059 - val_loss: 0.6314 - val_accuracy: 0.7798
Epoch 6/50
27606/27606 [==============================] - 165s 6ms/sample - loss: 0.4901 - accuracy: 0.8259 - val_loss: 0.6478 - val_accuracy: 0.7686
Epoch 7/50
27606/27606 [==========================

In [75]:
model.evaluate([test_images,test_images],test_labels, verbose=1)

3000/3000 [==============================] - 5s 2ms/sample - loss: 0.6149 - accuracy: 0.8020


[0.614936641295751, 0.802]